# High-level LSTM CNTK Example

In [1]:
print("Not working for some reason ...")

Not working for some reason ...


In [2]:
import numpy as np
import os
import sys
import cntk
from cntk.layers import Embedding, LSTM, Dense, Recurrence
from cntk import sequence
from common.params_lstm import *
from common.utils import *

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("CNTK: ", cntk.__version__)
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.13.1
CNTK:  2.1
GPU:  ['Tesla K80', 'Tesla K80', 'Tesla K80', 'Tesla K80']


In [4]:
def create_symbol():
    # Weight initialiser from uniform distribution
    # Activation (unless states) is None
    with cntk.layers.default_options(init = cntk.glorot_uniform()):
        x = Embedding(EMBEDSIZE)(features)
        # Fold same as recurrence but returns only final-state
        x = Recurrence(LSTM(NUMHIDDEN))(x)
        x = sequence.last(x)
        x = Dense(1)(x)
        return x

In [5]:
def init_model(m):
    # Loss (dense labels); check if support for sparse labels
    loss = cntk.cross_entropy_with_softmax(m, labels)  
    # ADAM
    #Alpha is the learning_rate
    #Beta1 is momentum parameter
    #Beta2 is variance_momentum parameter  
    learner = cntk.adam(m.parameters,
                        lr=cntk.learning_rate_schedule(LR, cntk.UnitType.minibatch) ,
                        momentum=cntk.momentum_schedule(BETA_1), 
                        variance_momentum=cntk.momentum_schedule(BETA_2),
                        epsilon=EPS,
                        unit_gain=False)
    trainer = cntk.Trainer(m, (loss, cntk.classification_error(m, labels)), [learner])
    return trainer

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = imdb_for_library(seq_len=MAXLEN, max_features=MAXFEATURES)# CNTK format
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Trimming to 20000 max-features
Padding to length 150
(25000, 150) (25000, 150) (25000,) (25000,)
float32 float32 float32 float32
CPU times: user 6.1 s, sys: 404 ms, total: 6.5 s
Wall time: 7.56 s


In [7]:
%%time
# Placeholders
features = sequence.input_variable(shape=MAXLEN, is_sparse=False)
labels = cntk.input_variable(1)

# Load symbol
sym = create_symbol()

print(features)
print(labels)

Input('Input3', [#, *], [150])
Input('Input4', [#], [1])
CPU times: user 0 ns, sys: 212 ms, total: 212 ms
Wall time: 275 ms


In [8]:
%%time
trainer = init_model(sym)

CPU times: user 48 ms, sys: 232 ms, total: 280 ms
Wall time: 372 ms


In [9]:
%%time
# Train model
for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        label = label.reshape(BATCHSIZE, 1)
        print(type(data))
        print(data.shape)
        print(type(label))
        print(label.shape)
        trainer.train_minibatch({features: data, labels: label})
        print(len(data))
        print(data[0])
        break
    break
    # Log (this is just last batch in epoch, not average of batches)
    eval_error = trainer.previous_minibatch_evaluation_average
    print("Epoch %d  |  Accuracy: %.6f" % (j+1, (1-eval_error)))

<class 'numpy.ndarray'>
(64, 150)
<class 'numpy.ndarray'>
(64, 1)


RuntimeError: Node 'Plus666' (Plus operation): TensorSliceWithMBLayoutFor: FrameRange's dynamic axis is inconsistent with data: {numTimeSteps:1, numParallelSequences:64, sequences:[{seqId:0, s:0, begin:0, end:1}, {seqId:1, s:1, begin:0, end:1}, {seqId:2, s:2, begin:0, end:1}, {seqId:3, s:3, begin:0, end:1}, {seqId:4, s:4, begin:0, end:1}, {seqId:5, s:5, begin:0, end:1}, {seqId:6, s:6, begin:0, end:1}, {seqId:7, s:7, begin:0, end:1}, {seqId:8, s:8, begin:0, end:1}, {seqId:9, s:9, begin:0, end:1}, {seqId:10, s:10, begin:0, end:1}, {seqId:11, s:11, begin:0, end:1}, {seqId:12, s:12, begin:0, end:1}, {seqId:13, s:13, begin:0, end:1}, {seqId:14, s:14, begin:0, end:1}, {seqId:15, s:15, begin:0, end:1}, {seqId:16, s:16, begin:0, end:1}, {seqId:17, s:17, begin:0, end:1}, {seqId:18, s:18, begin:0, end:1}, {seqId:19, s:19, begin:0, end:1}, {seqId:20, s:20, begin:0, end:1}, {seqId:21, s:21, begin:0, end:1}, {seqId:22, s:22, begin:0, end:1}, {seqId:23, s:23, begin:0, end:1}, {seqId:24, s:24, begin: